<a href="https://colab.research.google.com/github/JanEggers-hr/chatgpt-playground/blob/main/chatgpt_fuer_alle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# chatGPT für alle!

Ein Notebook, mit dem man (mit einem gespendeten API-Token) chatGPT nutzen kann, ohne sich anzumelden - und noch ein paar zusätzliche Einstellungen beeinflussen kann wie z.B. die Persönlichkeit, das verwendete Modell und die Parameter. 

Es wird empfohlen, eine Browser-Erweiterung wie den [Colab Automatic Clicker](https://addons.mozilla.org/en-US/firefox/addon/colab-automatic-clicker/) oder [Colab Auto Reconnect (Chrome)](https://chrome.google.com/webstore/detail/colab-auto-reconnect/ifilpgffgdbhafnaebocnofaehicbkem) zu nutzen, damit die Colab-Session offen bleibt. 

## Vorbereitungen

*Bitte einmal kurz auf die nächste Zelle klicken, um den Vorbereitungs-Code auszuführen: Einstell-Widgets erzeugen, die Library für die OpenAI-API laden.*

In [22]:
#@title
import requests
import json

# ipywidgets ist schon installiert
import ipywidgets as widgets
from IPython.display import display

# Modelle und Kosten definieren
# Kosten in US-Dollar je 1000 Tokens
models_pricing = {'gpt-3.5-turbo': 0.002,
          'text-davinci-003': 0.02,
          'gpt-4': 0.03,
          'gpt-4-32': 0.06}

textbox_api_key = widgets.Text(
    value='',
    placeholder='(...)',
    description='API-Token:',
)

area_system = widgets.Textarea(
    value = 'Du bist chatGPT, ein KI-Sprachsystem. Du bist freundlich \
und hilfsbereit und löst alle Aufgaben Schritt für Schritt.\n',
    rows=10,
    description = 'System:'
)

# Temperatur-Slider
slider_temperature = widgets.FloatSlider(
    value=0.7,
    min=0,
    max=1,
    step=0.1,
    description='Temperatur:',
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)

# Best-of-Slider
slider_bestof = widgets.IntSlider(
    value=1,
    min=1,
    max=4,
    description='Best Of:',
    orientation='horizontal',
    readout=True
)


dropdown_model = widgets.Dropdown(
    # Nimm die oben definierte Preisliste als Basis
    options=list(models_pricing.keys()),
    value=list(models_pricing.keys())[0],
    description='Modell:',
)

textbox_stop = widgets.Text(
    value='###\n',
    placeholder="###",
    description="Stop-Token:"
)

# Funktion wird bei Veränderung ausgeführt
def update_params(change):
    global temperature
    global api_key
    global system_prompt
    global model
    global stoptokens
    global best_of
    temperature = slider_temperature.value
    best_of = slider_bestof.value
    api_key = textbox_api_key.value
    system_prompt = area_system.value
    model = dropdown_model.value
    stoptokens = textbox_stop.value
    if (stoptokens == ""):
      stoptokens = None

# Verbinde die Widgets mit der Funktion zur Verarbeitung der Werte
textbox_api_key.observe(update_params, 'value')
slider_temperature.observe(update_params, 'value')
slider_bestof.observe(update_params, 'value')
area_system.observe(update_params, 'value')
textbox_stop.observe(update_params, 'value')
dropdown_model.observe(update_params, 'value')

# Bisschen breiter anzeigen
textbox_api_key.layout.width = '600px'
dropdown_model.layout.width = '300px'
area_system.layout.width = '600px'

# Vorbereitungen für die Einstellungen sind getan - jetzt die OpenAI-Libraries
update_params(0)
print("Widgets eingerichtet.")

# OpenAI-API-Library einbinden
!pip install -q openai
import openai
print("OpenAI-API-Library geladen.")


Widgets eingerichtet.
OpenAI-API-Library geladen.


## Der Chatbot

Damit man diesen Chatbot nutzen kann, **muss man ein gültiges API-Token in das entsprechende Feld kopieren.**

Die Parameter: 
- Die **Temperatur** bestimmt das Maß an Zufall, das das Sprachmodell nutzt - man könnte auch sagen: je höher die Temperatur, desto kreativer wird es.
- Das **Modell** bestimmt, welches Sprachmodell die Antwort berechnet - das derzeit mächtigste ist GPT-4. GPT-4 32k wird nur benötigt, wenn man sehr, sehr, sehr, lange Eingaben (bis ca. 25.000 Worte) verarbeiten muss.
- Das **Stop-Token** wird gebraucht, wenn ich die Eingaben strukturieren muss - etwa, wenn ich einzelne Beispiele für einen originalen und einen umgeschriebenen Text aufführe. 
- Das **System** beschreibt gewissermaßen die Persönlichkeit des Bots - das Sprachmodell orientiert sich bei seiner Antwort daran. (GPT3.5 nicht sehr stark.) 

In [23]:
#@title
from getpass import getpass
openai.api_key = getpass("OpenAI-API-Key eingeben: ")
previous_messages = []
spent_tokens = 0

def on_chatbot_reset_clicked(button):
    global previous_messages
    global spent_tokens
    previous_messages = []
    spent_tokens = 0
    chatbot_output_area.value = ''

# Die Einstellungs-Widgets anzeigen
# Setzt die globalen Variablen temperature, system_prompt, api_key, model, stoptokens
display(slider_temperature,
#        slider_bestof,
        dropdown_model,
        textbox_stop, 
        area_system)

# Hilfsfunktion : Kosten berechnen. 
def pricing(tokens):
    price = models_pricing.get(model)
    # Kosten in Dollar zurückgeben
    return(tokens * price / 1000)

def chatbot(prompts):
    # Prompt 
    response = openai.ChatCompletion.create(
        model=model,
        messages=prompts,
        n=1,
#        best_of = best_of,
        stop=stoptokens,
        temperature=temperature,
    )
    # Update the token usage widget
    update_token_usage_widget(response['usage']['total_tokens'])
    return response.choices[0].message["content"]

text_tokens = widgets.HTML(
    value = '<b>Verbrauchte Token</b>: 0 ($0.00)'
)

# Define the widget for displaying token usage
def update_token_usage_widget(value):
    global spent_tokens
    spent_tokens += value
    token_usage_text = f'<b>Verbrauchte Token:</b> Diesmal {value}, insgesamt {spent_tokens} ($ {pricing(value):.3f}) '
    text_tokens.value = token_usage_text

chatbot_output_area = widgets.HTML(
    value='',
    description='Dialog:',
    layout=widgets.Layout(width='100%')
)

# Define the function to be called when the chatbot is used
def on_chatbot_button_clicked(button):
    # Get the user's input and display it
    user_input = user_text.value
    user_text.value = ''
    # Generate a response from the chatbot
    chatbot_output_area.value += f'<p style="font-family: courier;"><b>Du</b>: {user_input}</p>'
    messages = [
            {"role": "system", "content": system_prompt},
            *previous_messages,
            {"role": "user", "content": user_input},
        ]
    chatbot_output = chatbot(messages)
    # Antwort anzeigen
    chatbot_output_area.value += '<p style="font-family: courier; font-style: italic;"><b>Chatbot: </b>' + chatbot_output + '</p>'
    previous_messages.extend([
        {"role": "user", "content": user_input},
        {"role": "assistant", "content": chatbot_output},
    ])

    

# Define the chatbot input and output widgets
user_text = widgets.Text(
    placeholder='...',
    description='Du:',
    layout=widgets.Layout(width='60%'),
)

# Definiere den Absenden-Button und binde ihn an on_chatbot_button_clicked
chatbot_button = widgets.Button(
    description='Absenden',
    layout=widgets.Layout(width='15%'),
)
chatbot_reset = widgets.Button(
    description = 'Reset',
    layout=widgets.Layout(width='15%')
)
chatbot_button.on_click(on_chatbot_button_clicked)
chatbot_reset.on_click(on_chatbot_reset_clicked)
# Abschicken auch durch Return in der user_text Box
user_text.on_submit(on_chatbot_button_clicked)

# Display the chatbot widgets
display(text_tokens)
display(chatbot_output_area, user_text, chatbot_button, chatbot_reset)


OpenAI-API-Key eingeben: ··········


FloatSlider(value=0.7, description='Temperatur:', max=1.0)

Dropdown(description='Modell:', layout=Layout(width='300px'), options=('gpt-3.5-turbo', 'text-davinci-003', 'g…

Text(value='###\n', description='Stop-Token:', placeholder='###')

Textarea(value='Du bist chatGPT, ein KI-Sprachsystem. Du bist freundlich und hilfsbereit und löst alle Aufgabe…

HTML(value='<b>Verbrauchte Token</b>: 0 ($0.00)')

HTML(value='', description='Dialog:', layout=Layout(width='100%'))

Text(value='', description='Du:', layout=Layout(width='60%'), placeholder='...')

Button(description='Absenden', layout=Layout(width='15%'), style=ButtonStyle())

Button(description='Reset', layout=Layout(width='15%'), style=ButtonStyle())